You should remove the `raise` exceptions below and insert your code in their place. The cells which say `DO NOT CHANGE THE CONTENT OF THIS CELL` are there to help you, if they fail, it's probably an indication of the fact that your code is wrong. You should not change their content - if you change them to make them correspond to what your program is producing, you will still not get the marks.

If you encounter an error while running your notebook that doesn't appear to be connected to RDDs (such as missing `imp`), you should check that you've run the initialization cells since you've started your latest cluster.

Before you turn your solution in, make sure everything runs as expected. With an attached cluster, you should **Clear State and Results** (under the **Clear** dropdown menu) and then click on the **Run all** icon. This runs all cells in the notebook from new. You should only submit this notebook if all cells run.

This homework is to be completed on your own. By the act of following these instructions and handing your work in, it is deemed that you have read and understand the rules on plagiarism as written in your student handbook.

# Character counting

This homework lets you practise writing functions along with using RDDs. The result will be a number of functions which you combine to take an input text and compute character frequencies in Spark.

The first four cells check that your Databricks setup is correct, download some files for you and move them to the right place if needed.

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
import sys
if not 'dbruntime.dbutils' in sys.modules.keys():
    import pyspark
    sc = pyspark.SparkContext()
    print("Unless you're grading this homework, you should be running this on Databricks.")

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
import os
import sys

file1 = "/FileStore/pride_and_prejudice.txt"
file2 = "/FileStore/madame_bovary.txt"
file3 = "/FileStore/polish_fairy_tales.txt"

os.environ['download_files'] = "false"

if 'dbruntime.dbutils' in sys.modules.keys():
    try:
        dbutils.fs.ls(file1)
    except:
        os.environ['download_files'] = "true"

In [0]:
%%bash
# DO NOT CHANGE THE CONTENT OF THIS CELL
if [ $download_files = "true" ]; then
    curl https://www.gutenberg.org/files/1342/1342-0.txt >/tmp/pride_and_prejudice.txt
    curl https://www.gutenberg.org/cache/epub/14155/pg14155.txt >/tmp/madame_bovary.txt    
    curl https://www.gutenberg.org/cache/epub/36668/pg36668.txt >/tmp/polish_fairy_tales.txt
fi

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
if 'dbruntime.dbutils' in sys.modules.keys():
    try:
        dbutils.fs.ls(file1)
    except:
        dbutils.fs.cp("file:/tmp/pride_and_prejudice.txt", file1)
        dbutils.fs.cp("file:/tmp/madame_bovary.txt", file2)
        dbutils.fs.cp("file:/tmp/polish_fairy_tales.txt", file3)

Write a function `to_lower` which takes a single line string and returns the string in lower case. So for the input "hElLO", `to_lower("hElLO")` should return `hello`.

In [0]:
def to_lower(s):
    # YOUR CODE HERE
    return s.lower()
  
to_lower('hElLO')

Out[153]: 'hello'

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert to_lower("hElLO WORLD!") == "hello world!", 'Unexpected lower case of hello world'
assert to_lower("123 RDDs") == "123 rdds", "Unexpected lower case of 123 RDDs"

Write a function `to_characters` which takes a single line string and returns a list of the (non space) characters contained within it. So for the input string "hello world", `to_characters("hello world")` should return `['h', 'e', 'l', 'l', 'o', 'w', 'o', 'r', 'l', 'd']`.

In [0]:
def to_characters(s):
    # YOUR CODE HERE
    return list(s.replace(' ',''))
  
to_characters('hello world')

Out[155]: ['h', 'e', 'l', 'l', 'o', 'w', 'o', 'r', 'l', 'd']

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert to_characters("hello world") == ['h', 'e', 'l', 'l', 'o', 'w', 'o', 'r', 'l', 'd'], "Unexpected hello world result"
assert to_characters("RDDs are fun!") == ['R', 'D', 'D', 's', 'a', 'r', 'e', 'f', 'u', 'n', '!'], "Unexpected RDDs are fun result"

Write the function `rdd_from_file` which takes a file path (such as `"/FileStore/pride_and_prejudice.txt"` or a variable which represents a file path, e.g. `file1`) as input and returns an RDD constructed by reading the lines into separate records.

In [0]:
def rdd_from_file(filename):
    # YOUR CODE HERE
    return sc.textFile(filename)

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
rdd1 = rdd_from_file(file1)
rdd2 = rdd_from_file(file2)
rdd3 = rdd_from_file(file3)

assert rdd1.count() == 14579, "Something has gone wrong with the RDD reading of file1"
assert rdd2.count() == 15609, "Something has gone wrong with the RDD reading of file2"
assert rdd3.count() == 3112, "Something has gone wrong with the RDD reading of file3"

Write the function `rdd_to_character_value_pair` which takes an RDD as input, lower cases each record, and splits it into characters (minus spaces) and makes each character into a pair with the second element `1` returning a list of such pairs. So for the input "Hello World", `line_to_character_value_pair` should return: `[('h', 1), ('e', 1), ('l', 1), ('l', 1), ('o', 1), ('w', 1), ('o', 1), ('r', 1), ('l', 1), ('d', 1)]`

This function can use the functions you have defined above (i.e. you should just need to construct pairs out of the character list returned by `to_characters` after `to_lower` has been used).

In [0]:
def rdd_to_character_value_pairs(rdd):
    # YOUR CODE HERE
    return rdd.map(lambda s: s.lower()).flatMap(lambda s: list(s.replace(' ',''))).map(lambda s: (s[0],1))

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
test_strings = ["hello WoRld", "RDDs are great fun!"]
testRDD = sc.parallelize(test_strings)
character_value_pairs = rdd_to_character_value_pairs(testRDD).collect()
assert character_value_pairs == [('h', 1), ('e', 1), ('l', 1), ('l', 1), ('o', 1), ('w', 1), ('o', 1), ('r', 1), ('l', 1), ('d', 1), ('r', 1), ('d', 1), ('d', 1), ('s', 1), ('a', 1), ('r', 1), ('e', 1), ('g', 1), ('r', 1), ('e', 1), ('a', 1), ('t', 1), ('f', 1), ('u', 1), ('n', 1), ('!', 1)], "Your function's output does not appear to match the specification"

Write a function `rdd_to_character_counts` which takes an RDD as input, creates pair RDDs of the format [(character, 1), .... ] and then adds up the 1s to get overall character counts. It should return an RDD containing (character, value) pairs where each character occurs only once and the value represents the frequency of the character in the original RDD.

Again, you can make use of previous functions you have created in this notebook.

In [0]:
def rdd_to_character_count(rdd):
    # YOUR CODE HERE
    return rdd_to_character_value_pairs(rdd).reduceByKey(lambda s1,s2: s1+s2)

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
from pyspark.rdd import RDD

count1 = rdd_to_character_count(rdd1)
assert isinstance(count1, RDD), "The output of your rdd_to_character_count is expected to be an RDD"

assert count1.lookup('s') == [33875], "Your counting may not be working as expected (rdd1 - s)"
assert count1.lookup('h') == [34661], "Your counting may not be working as expected (rdd1 - h)"
assert count1.lookup('a') == [42765], "Your counting may not be working as expected (rdd1 - a)"

count2 = rdd_to_character_count(rdd2)
count3 = rdd_to_character_count(rdd3)

assert isinstance(count2, RDD), "The output of your rdd_to_character_count is expected to be an RDD"
assert isinstance(count3, RDD), "The output of your rdd_to_character_count is expected to be an RDD"

assert count2.lookup('s') == [44074], "Your counting may not be working as expected (rdd2 - s)"
assert count2.lookup('h') == [6177], "Your counting may not be working as expected (rdd2 - h)"
assert count2.lookup('a') == [46611], "Your counting may not be working as expected (rdd2 - a)"

assert count3.lookup('s') == [5882], "Your counting may not be working as expected (rdd3 - s)"
assert count3.lookup('h') == [6856], "Your counting may not be working as expected (rdd3 - h)"
assert count3.lookup('a') == [7778], "Your counting may not be working as expected (rdd3 - a)"

Write a function `rdd_with_ordered_character_count` which takes an RDD, converts it into a pair RDD of the form `(character, numerical_value)` representing the frequency of each character (lower cased) in the input, and returns an RDD containing `(character, value)` pairs where the values are presented in decreasing order. So for the input "hEllO", the output should be `[(l, 2), (o, 1), (h, 1), (e, 1)]`. You should consider using the function(s) you have defined previously in this homework.

In [0]:
def rdd_with_ordered_character_count(rdd):
    # YOUR CODE HERE
    New = rdd_to_character_count(rdd)
    return New.map(lambda s: (s[1],s[0])).sortByKey(False).map(lambda s: (s[1],s[0]))

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
ordered1 = rdd_with_ordered_character_count(rdd1)
ordered2 = rdd_with_ordered_character_count(rdd2)
ordered3 = rdd_with_ordered_character_count(rdd3)

assert ordered1.take(10) == [('e', 71359), ('t', 48269), ('a', 42765), ('o', 41407), ('i', 38880), ('n', 38737), ('h', 34661), ('s', 33875), ('r', 33554), ('d', 22849)], "Your sorting may be going wrong"
assert ordered2.take(10) == [('e', 79764), ('a', 46611), ('s', 44074), ('t', 40678), ('i', 40356), ('n', 36296), ('r', 35857), ('l', 35195), ('u', 33588), ('o', 29092)], "Your sorting may be going wrong"
assert ordered3.take(10) == [('e', 13055), ('t', 8876), ('o', 7908), ('a', 7778), ('n', 7040), ('h', 6856), ('i', 6624), ('r', 6365), ('s', 5882), ('d', 4795)], "Your sorting may be going wrong"

Now you can compare the results to see how the order of most frequent characters differs between languages! (You shouldn't need to change the following cell.) `file1` was an English text, `file2` was in French and `file3` was Polish.

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
zipped = zip(ordered1.collect(), ordered2.collect(), ordered3.collect())
for values in zipped:
    print(values)

(('e', 71359), ('e', 79764), ('e', 13055))
(('t', 48269), ('a', 46611), ('t', 8876))
(('a', 42765), ('s', 44074), ('o', 7908))
(('o', 41407), ('t', 40678), ('a', 7778))
(('i', 38880), ('i', 40356), ('n', 7040))
(('n', 38737), ('n', 36296), ('h', 6856))
(('h', 34661), ('r', 35857), ('i', 6624))
(('s', 33875), ('l', 35195), ('r', 6365))
(('r', 33554), ('u', 33588), ('s', 5882))
(('d', 22849), ('o', 29092), ('d', 4795))
(('l', 22068), ('d', 20056), ('l', 3997))
(('u', 15507), ('c', 16344), ('u', 2886))
(('m', 15113), ('m', 15786), ('w', 2702))
(('c', 14136), ('p', 14651), ('g', 2466))
(('y', 13052), (',', 12533), ('c', 2432))
(('w', 12595), ('v', 8791), ('m', 2386))
(('f', 12376), ('é', 8295), ('f', 2272))
(('g', 10450), ("'", 7465), ('y', 1986))
(('b', 9361), ('.', 6448), (',', 1966))
((',', 9291), ('b', 6245), ('p', 1934))
(('p', 8733), ('h', 6177), ('b', 1636))
(('.', 6403), ('f', 5880), ('.', 1071))
(('v', 5842), ('q', 5709), ('k', 919))
(('k', 3347), ('g', 5190), ('v', 847))
(('“', 1

In fact, we can do some pretty visualizations. The following code requires the `letter_frequencies.txt` file (downloadable from Blackboard) to be uploaded to `/FileStore/tables/`

This file contains the letter frequency information derived from all the languages of Wikipedia available from [http://simia.net/letters/]

The goal of this section is:
* To ensure that you're happy uploading files
* Meet visualizations
* Show you the power of python
* Give you some more example code

In the following cell, please set a variable named `my_native_language` to your native language - if the spelling doesn't match that on the Simia page, the visualization for your language will not work. If information is not available for your language, you will still be able to see the visualizations for the languages examined above.

In [0]:
# Expected format: my_native_language = "MyLanguage"
# YOUR CODE HERE
myLanguage = sc.textFile("dbfs:/FileStore/tables/letter_frequencies.txt")
my_native_language = "MyLanguage"

In [0]:
try:
    assert isinstance(my_native_language,str), "my_native_language should be set to a string corresponding to your native language"
    assert my_native_language.strip() != ""
except:
    assert False, "You need to define the variable my_native_language and set it to your native language"

Now we check that the `letter_frequencies.txt` file is present in `/FileStore/tables` and extract a list of languages it contains into a variable called `languages`.

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL

# Check the file is present as expected
if 'dbruntime.dbutils' in sys.modules.keys():
    try:
        dbutils.fs.ls("/FileStore/tables/letter_frequencies.txt")
    except:
        assert False, "It is assumed that you've put the /FileStore/tables/letter_frequencies.txt file in the DBFS /FileStore/tables"    
    
# Read contents of the file into an RDD
letterFreqRDD = sc.textFile("/FileStore/tables/letter_frequencies.txt")
# Turn each record into a pair RDD, normalizing language names
langFreqRDD = letterFreqRDD.map(lambda x: (x.split(',')[0].lower(), x.split(',')[1]))
# Extract languages
languagesRDD = langFreqRDD.keys()
# Convert into a non-parallelized list
languages = languagesRDD.collect()

Now we perform the lookup of the frequencies for the three languages investigated above and the native language (if different).

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
# Normalize native language name
my_native_language = my_native_language.lower().strip()
# Look up the freqs for the native language if possible (and different)
native_html = ""
if my_native_language in languages and not my_native_language in ["english","french","polish"]:
    native_html = langFreqRDD.lookup(my_native_language)[0]
# Look up English freqs
english_html = langFreqRDD.lookup('english')[0]
# Look up French freqs
french_html = langFreqRDD.lookup('french')[0]
# Look up Polish freqs
polish_html = langFreqRDD.lookup('polish')[0]

The frequency information is encoded in an HTML string. Python has a module which can render this as part of the notebook, so we use that.

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
from IPython.core.display import HTML
if not native_html == "":
    display(HTML('<tr><td><strong>' + my_native_language.capitalize() + ':</strong></td>' + native_html))
display(HTML('<tr><td><strong>English:</strong></td>' + english_html))
display(HTML('<tr><td><strong>French:</strong></td>' + french_html))
display(HTML('<tr><td><strong>Polish:</strong></td>' + polish_html))

English:


French:


Polish:
